इस लेख में शामिल जप्टर नोटबुक [अध्याय 4 कोड बेस](https://github.com/datawhalechina/learn-nlp-with-transformers/tree/main/docs/%E7%AF%87%E7%) में है AB%A04-%E4%BD%BF%E7%94%A8Transformers%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1).

प्रासंगिक डेटासेट और मॉडल को तुरंत डाउनलोड करने के लिए इस ट्यूटोरियल को सीधे Google कोलाब नोटबुक का उपयोग करके खोलने की अनुशंसा की जाती है।
यदि आप इस नोटबुक को Google Colab में खोल रहे हैं, तो आपको ट्रांसफॉर्मर और 🤗Datasets लाइब्रेरीज़ को इंस्टॉल करने की आवश्यकता हो सकती हैस्थापित करने के लिए निम्न आदेश दें।

In [1]:
# !पिप डेटासेट ट्रांसफार्मर स्थापित करें

# मशीन प्रश्न उत्तर कार्य पर ट्रांसफार्मर मॉडल को फाइन-ट्यून करें

इस नोटबुक में, हम सीखेंगे कि मशीन प्रश्न उत्तर कार्यों को हल करने के लिए [🤗 ट्रांसफॉर्मर्स] (https://github.com/huggingface/transformers) के ट्रांसफार्मर मॉडल को कैसे ठीक किया जाए। यह लेख मुख्य रूप से निष्कर्षण प्रश्न उत्तर कार्य को हल करता है: एक प्रश्न और एक पाठ दिया गया है, पाठ खंड (स्पैन) खोजें जो पाठ से प्रश्न का उत्तर दे सके। 'ट्रेनर' एपीआई और डेटासेट पैकेज का उपयोग करके, हम आसानी से डेटासेट लोड करेंगे और फिर ट्रांसफार्मर को ठीक करेंगे नीचे दिया गया चित्र एक सरल उदाहरण देता हैउदाहरण
![QA कार्य का प्रतिनिधित्व करने वाला विजेट अनुमान](images/question_answering.png)

**नोट:** नोट: इस लेख में प्रश्न का उत्तर देने का कार्य पाठ से उत्तर निकालना है, न कि सीधे उत्तर उत्पन्न करना!

इस नोटबुक में डिज़ाइन किए गए उदाहरणों का उपयोग SQUAD 1 और SQUAD 2 के समान किसी भी निष्कर्षित प्रश्न उत्तर कार्य को हल करने के लिए किया जा सकता है, और जब तक [मॉडल हब] (https://huggingface.co/models) में किसी भी मॉडल चेकपॉइंट का उपयोग किया जा सकता है इन मॉडलों में एक टोकन वर्गीकरण प्रमुख और एक तेज़ टोकनाइज़र होता है। मॉडल और तेज़ टोकनाइज़र के बीच पत्राचार के लिए, देखें: [यह तालिका] (https://huggingface.co/transformers/index.html#bigtable)।

यदि आपका डेटासेट इस नोटबुक से भिन्न है, तो आप इस नंबर का उपयोग कर सकते हैंकेवल मामूली समायोजन के साथ सीधे टीबुक करें, आपके हार्डवेयर (कंप्यूटर मेमोरी, ग्राफिक्स कार्ड आकार) के आधार पर, आपको आउट-ऑफ-मेमोरी त्रुटियों से बचने के लिए बैच आकार को उचित रूप से समायोजित करने की आवश्यकता है।
उन तीन मापदंडों को सेट करें, फिर बाकी नोटबुक सुचारू रूप से चलनी चाहिए:

In [ ]:
# स्क्वाड_v2 क्रमशः SQUAD v1 या SQUAD v2 का उपयोग करने के लिए सही या गलत के बराबर है।
# यदि आप अन्य डेटासेट का उपयोग कर रहे हैं, तो सत्य का अर्थ है कि मॉडल "अनिवार्य" प्रश्नों का उत्तर दे सकता है, अर्थात, कुछ प्रश्न उत्तर नहीं देते हैं, जबकि गलत का अर्थ है कि सभी प्रश्नों का उत्तर दिया जाना चाहिए।
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## डेटासेट लोड करें

हम डेटा डाउनलोड करने और हमारे लिए आवश्यक मूल्यांकन संकेतक (बेंचमार्क की तुलना में) प्राप्त करने के लिए [🤗 डेटासेट] (https://github.com/huggingface/datasets) लाइब्रेरी का उपयोग करेंगे।

इन दोनों कार्यों को `load_dataset` और `load_metric` फ़ंक्शन का उपयोग करके आसानी से पूरा किया जा सकता है।

In [ ]:
from datasets import load_dataset, load_metric

उदाहरण के तौर पर, हम इस नोटबुक में [SQUAD डेटासेट](https://rajpurkar.github.io/SQuAD-explorer/) का उपयोग करेंगे। इसी तरह, यह नोटबुक डेटासेट रिपॉजिटरी द्वारा प्रदान किए गए सभी प्रश्न उत्तर डेटासेट के साथ भी संगत है।

यदि आप अपने स्वयं के डेटासेट (json या csv प्रारूप) का उपयोग कर रहे हैं, तो लोड करने का तरीका जानने के लिए कृपया [डेटासेट दस्तावेज़] (https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) देखें आपका कस्टम डेटासेट। आपको प्रत्येक कॉलम के लिए उपयोग किए गए नामों को समायोजित करने की आवश्यकता हो सकती हैउम्म.

In [ ]:
# डेटा डाउनलोड करें (सुनिश्चित करें कि नेटवर्क है)
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

इसके अलावा, आप हमारे द्वारा प्रदान किए गए [लिंक] (https://gas.graviti.cn/dataset/datawhale/SQuAD) से डेटा भी डाउनलोड कर सकते हैं और इसे डीकंप्रेस कर सकते हैं, दो डीकंप्रेस्ड json फ़ाइलों को `docs/Chapter 4 में कॉपी कर सकते हैं -एनएलपी कार्यों/डेटासेट/स्क्वाड निर्देशिका को हल करने के लिए ट्रांसफॉर्मर का उपयोग करना, और फिर इसे निम्नलिखित कोड के साथ लोड करना।

In [ ]:
import os

data_path = './dataset/squad/'
path = os.path.join(data_path, 'squad.py')
cache_dir = os.path.join(data_path, 'cache')
data_files = {"train": os.path.join(data_path, "train-v1.1.json"), "validation": os.path.join(data_path, "dev-v1.1.json")}
datasets = load_dataset(path, data_files=data_files, cache_dir=cache_dir)

यह `डेटासेट` ऑब्जेक्ट एक [`डेटासेटडिक्ट`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) संरचना है, और प्रशिक्षण, सत्यापन और परीक्षण क्रमशः इस निर्देश में एक कुंजी के अनुरूप हैं .

In [ ]:
# निम्नलिखित डेटासेट और उनके गुण देखें
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

चाहे वह एक प्रशिक्षण सेट, सत्यापन सेट या परीक्षण सेट हो, प्रत्येक प्रश्न-उत्तर डेटा नमूने में तीन कुंजी होंगी: "संदर्भ", "प्रश्न" और "उत्तर"।

हम किसी नमूने का चयन करने के लिए सबस्क्रिप्ट का उपयोग कर सकते हैं।

In [ ]:
datasets["train"][0]
# उत्तर उत्तर दर्शाते हैं
# संदर्भ एक पाठ खंड का प्रतिनिधित्व करता है
# प्रश्न प्रश्न का प्रतिनिधित्व करता है

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

उत्तरों के एनोटेशन पर ध्यान दें। टेक्स्ट स्निपेट में उत्तर टेक्स्ट देने के अलावा, उत्तर उत्तर की स्थिति भी बताते हैं (वर्ण की शुरुआत से गिना जाता है, उपरोक्त उदाहरण में यह 515वां स्थान है)।

यह समझने के लिए कि डेटा कैसा दिखता है, निम्न फ़ंक्शन प्रदर्शन के लिए डेटासेट से यादृच्छिक रूप से कुछ उदाहरणों का चयन करेगा।

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"], num_examples=2)

,answers,context,id,question,title
0,"{'answer_start': [185], 'text': ['diesel fuel']}","In Alberta, five bitumen upgraders produce synthetic crude oil and a variety of other products: The Suncor Energy upgrader near Fort McMurray, Alberta produces synthetic crude oil plus diesel fuel; the Syncrude Canada, Canadian Natural Resources, and Nexen upgraders near Fort McMurray produce synthetic crude oil; and the Shell Scotford Upgrader near Edmonton produces synthetic crude oil plus an intermediate feedstock for the nearby Shell Oil Refinery. A sixth upgrader, under construction in 2015 near Redwater, Alberta, will upgrade half of its crude bitumen directly to diesel fuel, with the remainder of the output being sold as feedstock to nearby oil refineries and petrochemical plants.",571b074c9499d21900609be3,"Besides crude oil, what does the Suncor Energy plant produce?",Asphalt
1,"{'answer_start': [191], 'text': ['the GIOVE satellites for the Galileo system']}","Compass-M1 is an experimental satellite launched for signal testing and validation and for the frequency filing on 14 April 2007. The role of Compass-M1 for Compass is similar to the role of the GIOVE satellites for the Galileo system. The orbit of Compass-M1 is nearly circular, has an altitude of 21,150 km and an inclination of 55.5 degrees.",56e1161ccd28a01900c6757b,The purpose of the Compass-M1 satellite is similar to the purpose of what other satellite?,BeiDou_Navigation_Satellite_System


## प्रशिक्षण डेटा को प्रीप्रोसेस करना

मॉडल में डेटा फीड करने से पहले, हमें डेटा को प्रीप्रोसेस करना होगा। प्रीप्रोसेसिंग टूल को 'टोकनाइज़र' कहा जाता है। पहले इनपुट को टोकनाइज़ करता है, फिर टोकन को प्री-मॉडल में आवश्यक संबंधित टोकन आईडी में परिवर्तित करता है। उन्हें मॉडल द्वारा आवश्यक इनपुट प्रारूप में परिवर्तित करता है।

डेटा प्रीप्रोसेसिंग के उद्देश्य को प्राप्त करने के लिए, हम अपने टोकननाइज़र को इंस्टेंट करने के लिए `AutoTokenizer.from_pretrained` विधि का उपयोग करते हैं, जो सुनिश्चित करता है:

- हमें एक टोकननाइज़र मिलता है जो टी से मेल खाता हैo पूर्व-प्रशिक्षित मॉडल को एक-एक करके।
- निर्दिष्ट मॉडल चेकपॉइंट के अनुरूप टोकननाइज़र का उपयोग करते समय, हम मॉडल के लिए आवश्यक शब्दावली, अधिक सटीक रूप से, टोकन शब्दावली भी डाउनलोड करते हैं।

इस डाउनलोड की गई टोकन शब्दावली को कैश कर दिया जाएगा ताकि दोबारा उपयोग करने पर इसे दोबारा डाउनलोड न किया जा सके।

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

निम्नलिखित कोड के लिए आवश्यक है कि टोकननाइज़र ट्रांसफार्मर प्रकार का होना चाहिए। PreTrainedTokenizerFast, क्योंकि हमें प्रीप्रोसेसिंग के दौरान फास्ट टोकननाइज़र (जैसे मल्टी-थ्रेडेड फास्ट टोकननाइज़र) की कुछ विशेष सुविधाओं का उपयोग करने की आवश्यकता होती है।

मॉडल के अनुरूप लगभग सभी टोकननाइज़र में तेज़ टोकननाइज़र होते हैं। हम [मॉडल टोकनाइज़र कॉरेस्पोंडेंस टेबल] (https://huggingface.co/transformers/index.html#bigtable) में सभी पूर्व-प्रशिक्षित मॉडल के अनुरूप टोकननाइज़र की विशेषताओं को देख सकते हैं। ).

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
# यदि हम टोकननाइज़र प्रीप्रोसेसिंग के बाद टेक्स्ट फॉर्मेट देखना चाहते हैं, तो हम केवल टोकननाइज़र की टोकननाइज़ विधि का उपयोग करते हैं, विशेष टोकन जोड़ने का अर्थ है प्री-ट्रेनिंग मॉडल के लिए आवश्यक विशेष टोकन जोड़ना।
print("单个文本tokenize: {}".format(tokenizer.tokenize("What is your name?"), add_special_tokens=True))
print("2个文本tokenize: {}".format(tokenizer.tokenize("My name is Sylvain.", add_special_tokens=True)))
# पूर्व-प्रशिक्षित मॉडल के इनपुट प्रारूप के लिए टोकन आईडी और एक एटेटनियन मास्क की आवश्यकता होती है। पूर्व-प्रशिक्षित मॉडल प्रारूप के लिए आवश्यक इनपुट प्राप्त करने के लिए आप निम्न विधि का उपयोग कर सकते हैं।

单个文本tokenize: ['what', 'is', 'your', 'name', '?']
2个文本tokenize: ['[CLS]', 'my', 'name', 'is', 'sy', '##lva', '##in', '.', '[SEP]']


टोकननाइज़र एकल टेक्स्ट या टेक्स्ट की एक जोड़ी को प्रीप्रोसेस कर सकता है। टोकननाइज़र प्रीप्रोसेसिंग के बाद प्राप्त डेटा पूर्व-प्रशिक्षित मॉडल के इनपुट प्रारूप से मिलता है।

In [ ]:
# एकल पाठ को प्रीप्रोसेस करें
tokenizer("What is your name?")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# दो टेक्स्ट को प्रीप्रोसेस करें। आप देख सकते हैं कि टोकनाइज़र शुरुआत में 101 टोकन आईडी जोड़ता है, दो टेक्स्ट को अलग करने के लिए बीच में 102 टोकन आईडी जोड़ता है, और 102 के साथ समाप्त होता है। ये नियम पूर्व-प्रशिक्षित मॉडल द्वारा डिज़ाइन किए गए हैं।
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

ऊपर देखे गए टोकन आईडी या इनपुट_आईडी आमतौर पर पूर्व-प्रशिक्षित मॉडल के नाम के साथ भिन्न होते हैं। इसका कारण यह है कि अलग-अलग पूर्व-प्रशिक्षित मॉडल पूर्व-प्रशिक्षण के दौरान अलग-अलग नियम निर्धारित करते हैं वही, टोकननाइज़र द्वारा पूर्व-संसाधित इनपुट प्रारूप मॉडल आवश्यकताओं को पूरा करेगा। पूर्व-प्रसंस्करण के बारे में अधिक जानकारी के लिए, कृपया [यह ट्यूटोरियल](https://huggingface.co/transformers/preprocessing.html) देखें।

अब हमें भी इसके बारे में सोचने की जरूरत हैपूर्व-प्रशिक्षित मशीन प्रश्न उत्तर देने वाले मॉडल बहुत लंबे टेक्स्ट को कैसे संभालते हैं, सामान्यतया, पूर्व-प्रशिक्षित मॉडल इनपुट के लिए अधिकतम लंबाई की आवश्यकता होती है, इसलिए हम आमतौर पर ओवरलॉन्ग इनपुट को छोटा कर देते हैं, हालांकि, यदि हम प्रश्न और उत्तर में ओवरलॉन्ग संदर्भ को छोटा करते हैं डेटा ट्रिपल <प्रश्न, संदर्भ, उत्तर>, हम उत्तर खो सकते हैं (क्योंकि हम उत्तर के रूप में संदर्भ से एक छोटा सा टुकड़ा निकालते हैं), निम्नलिखित कोड एक उदाहरण ढूंढता है जो लंबाई और वें से अधिक हैएन आपको दिखाता है कि इसे कैसे संभालना है। हम लंबे इनपुट को कई छोटे इनपुट में काटते हैं, जिनमें से प्रत्येक को मॉडल की अधिकतम लंबाई इनपुट आवश्यकता को पूरा करना होगा क्योंकि उत्तर स्लाइस के समान स्थान पर हो सकता है, हमें आसन्न के बीच चौराहे की अनुमति देने की आवश्यकता है स्लाइस, जिसे कोड में `doc_stride` पैरामीटर द्वारा नियंत्रित किया जाता है..

मशीन प्रश्न उत्तर देने वाले पूर्व-प्रशिक्षण मॉडल आमतौर पर प्रश्न और संदर्भ को इनपुट के रूप में जोड़ते हैं, और फिर मॉडल को संदर्भ से उत्तर ढूंढने देते हैं।

In [ ]:
max_length = 384 # 输入feature的最大长度，question和context拼接之后
doc_stride = 128 # 2个切片之间的重合token数量。

लूप के लिए डेटासेट पर पुनरावृत्ति होती है, जो एक अत्यंत लंबे नमूने की तलाश करता है। नोटबुक उदाहरण मॉडल के लिए आवश्यक अधिकतम इनपुट 384 है (512 भी आमतौर पर उपयोग किया जाता है)।

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

यदि छोटा नहीं किया गया है, तो इनपुट लंबाई 396 है

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

396

अब यदि हम 384 की अधिकतम लंबाई तक काट-छांट करते हैं, तो अतिरिक्त-लंबे भाग की जानकारी खो जाएगी।

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

ध्यान दें कि, आम तौर पर बोलते हुए, हम केवल संदर्भ को काटते हैं, प्रश्न को नहीं। चूँकि संदर्भ प्रश्न के बाद संक्षिप्त होता है और दूसरे पाठ से मेल खाता है, हम इसे नियंत्रित करने के लिए `only_third` का उपयोग करते हैं। ओवरलैप को नियंत्रित करने के लिए टोकनाइज़र `doc_stride` का उपयोग करता है स्लाइस के बीच की लंबाई.

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

चूंकि ओवरलांग इनपुट को काटा जाता है, हमें कई इनपुट मिलते हैं, इन इनपुट_आईडी की लंबाई होती है

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 157]

हम पूर्व-संसाधित टोकन आईडी, इनपुट_आईडी को टेक्स्ट प्रारूप में पुनर्स्थापित कर सकते हैं:

In [ ]:
for i, x in enumerate(tokenized_example["input_ids"][:2]):
    print("切片: {}".format(i))
    print(tokenizer.decode(x))

切片: 0
[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth a

चूंकि हमने बहुत लंबे पाठ को काट दिया है, इसलिए हमें उत्तर की स्थिति फिर से ढूंढनी होगी (संदर्भ के प्रत्येक भाग की शुरुआत के सापेक्ष)। मशीन प्रश्न उत्तर देने वाला मॉडल उत्तर की स्थिति (प्रारंभ और अंत) का उपयोग करेगा प्रशिक्षण लेबल के रूप में उत्तर, प्रारंभ और अंत की स्थिति (उत्तर के टोकन आईडीएस के बजाय), स्लाइस को मूल इनपुट के साथ संबंधित संबंध होना चाहिए, और स्लाइसिंग के बाद संदर्भ में प्रत्येक टोकन की स्थिति होनी चाहिए। पत्र-व्यवहारमूल सुपर लॉन्ग संदर्भ में स्थिति के लिए डीएस, टोकननाइज़र में, आप इस पत्राचार का नक्शा प्राप्त करने के लिए `रिटर्न_ऑफसेट्स_मैपिंग` पैरामीटर का उपयोग कर सकते हैं:

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
# स्लाइस के पहले और बाद में स्थिति सबस्क्रिप्ट के बीच पत्राचार प्रिंट करें
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

ऊपर जो मुद्रित किया गया है वह मूल संदर्भ स्लाइस मेंटोकनाइज्ड_उदाहरण के 0वें स्लाइस के पहले 100 टोकन की स्थिति है। ध्यान दें कि पहला टोकन `[सीएलएस]` (0, 0) पर सेट है क्योंकि यह टोकन इसका हिस्सा नहीं है। प्रश्न या उत्तर। दूसरे टोकन के अनुरूप शुरुआती और अंतिम स्थिति 0 और 3 हैं। हम स्लाइस करने के बाद टोकन आईडी के अनुसार संबंधित टोकन को परिवर्तित कर सकते हैं, फिर स्लाइस करने से पहले टोकन स्थिति पर वापस मैप करने के लिए `offset_mapping` पैरामीटर का उपयोग कर सकते हैं; वें खोजेंई टोकन मूल स्थिति में हैं क्योंकि प्रश्न संदर्भ के सामने जुड़ा हुआ है, आप इसे केवल सबस्क्रिप्ट के अनुसार प्रश्न से पा सकते हैं।

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


इसलिए, हमें स्लाइस से पहले और बाद में स्थिति पत्राचार मिलता है। हमें प्रश्न और संदर्भ के बीच अंतर करने के लिए `sequence_ids` पैरामीटर का भी उपयोग करने की आवश्यकता है।

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

'कोई नहीं' विशेष टोकन से मेल खाता है, और 0 या 1 क्रमशः पहले पाठ और दूसरे पाठ का प्रतिनिधित्व करता है। चूंकि हम प्रश्न को पहले इनपुट के रूप में और संदर्भ को दूसरे इनपुट के रूप में पास करते हैं, वे क्रमशः प्रश्न और संदर्भ के अनुरूप होते हैं प्रीप्रोसेसिंग के बाद फीचर्स में एनोटेटेड उत्तर की स्थिति पा सकते हैं:

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# वर्तमान टेक्स्ट का स्टार्ट टोकन इंडेक्स ढूंढें।
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# वर्तमान पाठ का अंतिम टोकन आईडीनेक्स ढूंढें।
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# जांचें कि क्या उत्तर पाठ अंतराल के बाहर है, इस मामले में, इसका मतलब है कि नमूने का डेटा सीएलएस टोकन स्थिति पर चिह्नित है।
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
# टोकन_स्टार्ट_इंडेक्स और टोकन_एंड_इंडेक्स को उत्तर के दोनों ओर ले जाएं।
# नोट: उत्तर अंत में सीमा स्थिति में है।
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print("start_position: {}, end_position: {}".format(start_position, end_position))
else:
    print("The answer is not in this feature.")

start_position: 23, end_position: 26


हमें उत्तर के स्थान को सत्यापित करने की आवश्यकता है। सत्यापन विधि है: उत्तर स्थान सूचकांक का उपयोग करें, संबंधित टोकन आईडी प्राप्त करें, फिर इसे पाठ में परिवर्तित करें, और फिर इसकी तुलना मूल उत्तर से करें।

In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600


कभी-कभी प्रश्न को संदर्भ के साथ जोड़ा जाता है, और कभी-कभी संदर्भ को प्रश्न के साथ जोड़ा जाता है। विभिन्न मॉडलों की अलग-अलग आवश्यकताएं होती हैं, इसलिए हमें इसे निर्दिष्ट करने के लिए `पैडिंग_साइड` पैरामीटर का उपयोग करने की आवश्यकता होती है।

In [ ]:
pad_on_right = tokenizer.padding_side == "right" #context在右边

अब, आइए सभी चरणों को एक साथ संयोजित करें। ऐसे मामले के लिए जहां संदर्भ में कोई उत्तर नहीं है, हम सीधे चिह्नित उत्तर प्रारंभ और अंत स्थिति को सीएलएस की सबस्क्रिप्ट पर रखते हैं। यदि पैरामीटर `allow_impossible_answers` `गलत` है, तो ये बिना उत्तर वाले नमूनों को फेंक दिया जाएगा सरलता के लिए, हम पहले उन्हें फेंक देंगे।

In [ ]:
def prepare_train_features(examples):
# हमें सभी जानकारी को बनाए रखते हुए उदाहरणों पर ट्रंकेशन और पैडिंग करने की आवश्यकता है, इसलिए हमें स्लाइसिंग विधि का उपयोग करने की आवश्यकता है।
# प्रत्येक सुपर लंबे टेक्स्ट उदाहरण को कई इनपुट में विभाजित किया जाएगा, और दो आसन्न इनपुट के बीच प्रतिच्छेदन होगा।
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

# हम स्लाइस आईडी को मूल आईडी में मैप करने के लिए ओवरफ़्लो_टू_सैंपल_मैपिंग पैरामीटर का उपयोग करते हैं।
# उदाहरण के लिए, यदि 2 उदाहरणों को 4 स्लाइस में काटा जाता है, तो संबंधित स्लाइस [0, 0, 1, 1] होते हैं, और पहले दो स्लाइस मूल पहले उदाहरण के अनुरूप होते हैं।
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
# ऑफसेट_मैपिंग भी 4 स्लाइस से मेल खाती है
# ऑफसेट_मैपिंग पैरामीटर हमें मूल इनपुट पर मैप करने में मदद करता है क्योंकि उत्तर मूल इनपुट पर अंकित है, यह हमें उत्तर की शुरुआती और अंतिम स्थिति ढूंढने में मदद करता है।
    offset_mapping = tokenized_examples.pop("offset_mapping")

# डेटा को पुनः लेबल करें
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
# प्रत्येक टुकड़े को संसाधित करें
# बिना उत्तर वाले नमूनों को सीएलएस पर लेबल करें
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

# प्रश्न और संदर्भ के बीच अंतर करना
        sequence_ids = tokenized_examples.sequence_ids(i)

# मूल उदाहरण सूचकांक प्राप्त करें।
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
# यदि कोई उत्तर नहीं है, तो उत्तर के रूप में सीएलएस के स्थान का उपयोग करें।
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
# उत्तर की वर्ण स्तरीय आरंभ/समाप्ति स्थिति.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

# टोकन स्तर पर सूचकांक प्रारंभ खोजें।
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

# टोकन लेवल इंडेक्स का अंत ढूंढें।
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

# जांचें कि क्या उत्तर पाठ की लंबाई से अधिक है। यदि यह अधिक है, तो सीएलएस इंडेक्स का उपयोग मार्कर के रूप में भी किया जाता है।
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
# यदि यह अधिक नहीं है, तो उत्तर टोकन की आरंभ और समाप्ति स्थिति ज्ञात करें।
# नोट: यदि उत्तर अंतिम शब्द (एज केस) है तो हम अंतिम ऑफसेट के बाद जा सकते हैं।
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

उपरोक्त प्रीप्रोसेसिंग फ़ंक्शन एक नमूने या एकाधिक नमूना उदाहरणों को संसाधित कर सकता है। यदि यह एकाधिक नमूनों को संसाधित करता है, तो यह एकाधिक नमूनों के प्रीप्रोसेसिंग के परिणामों की एक सूची लौटाता है।

In [ ]:
features = prepare_train_features(datasets['train'][:5])
#प्रक्रिया 5 नमूने

इसके बाद, सभी नमूनों पर प्रीप्रोसेसिंग फ़ंक्शन 'prepare_train_features' को लागू करने के लिए 'मैप' फ़ंक्शन का उपयोग करके डेटासेट डेटासेट में सभी नमूनों को प्रीप्रोसेस किया जाता है।

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

इससे भी बेहतर, अगली बार संसाधित होने पर पुनर्गणना से बचने के लिए लौटाए गए परिणाम स्वचालित रूप से कैश किए जाते हैं (लेकिन ध्यान रखें कि यदि इनपुट बदलता है, तो यह कैश से प्रभावित हो सकता है!) डेटासेट लाइब्रेरी फ़ंक्शन यह निर्धारित करने के लिए इनपुट पैरामीटर का पता लगाएगा यदि कोई परिवर्तन हैं, तो कैश्ड डेटा का उपयोग किया जाएगा। यदि परिवर्तन हैं, तो डेटा को पुन: संसाधित किया जाएगा। हालांकि, यदि इनपुट पैरामीटर नहीं बदलते हैं, तो जब आप चाहें तो कैश को साफ़ करना सबसे अच्छा है चांगई इनपुट। इसे साफ़ करने का तरीका `load_from_cache_file=False` पैरामीटर का उपयोग करना है। इसके अलावा, ऊपर इस्तेमाल किया गया `batched=True` पैरामीटर टोकननाइज़र की एक विशेषता है, क्योंकि यह इनपुट को संसाधित करने के लिए कई थ्रेड्स का उपयोग करेगा। समानांतर।

## मॉडल को फाइन-ट्यूनिंग करना

वर्तमान में, हमने प्रशिक्षण/फाइन-ट्यूनिंग के लिए आवश्यक डेटा को प्रीप्रोसेस किया है। अब हम पूर्व-प्रशिक्षित मॉडल डाउनलोड करते हैं। चूंकि हम मशीन प्रश्न उत्तर देने वाले कार्य कर रहे हैं, इसलिए हम इस क्लास `ऑटोमॉडलफॉरक्वेश्चनएन्सवरिंग` का उपयोग करते हैं। यह मॉडल भी टोकननाइज़र के समान है `from_pretrained` विधि का उपयोग करके लोड किया गया।

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 268M/268M [00:46<00:00, 5.79MB/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs

चूंकि जिस कार्य को हम ठीक कर रहे हैं वह मशीन प्रश्न उत्तर देने वाला कार्य है, और हम एक पूर्व-प्रशिक्षित भाषा मॉडल लोड कर रहे हैं, उपरोक्त हमें संकेत देगा कि मॉडल लोड करते समय कुछ बेजोड़ तंत्रिका नेटवर्क पैरामीटर हटा दिए गए थे (तंत्रिका नेटवर्क प्रमुख) पूर्व-प्रशिक्षित भाषा मॉडल को फेंक दिया गया था, और मशीन के प्रश्न का उत्तर देने वाले तंत्रिका नेटवर्क प्रमुख को बेतरतीब ढंग से आरंभ किया गया था)।

इन बेतरतीब ढंग से आरंभ किए गए मापदंडों के कारण, हमें नए डेटा पर अपने मॉडल को फिर से ठीक करने की आवश्यकता हैएट.

`ट्रेनर` प्रशिक्षण उपकरण प्राप्त करने के लिए, हमें 3 और तत्वों की आवश्यकता है, जिनमें से सबसे महत्वपूर्ण है प्रशिक्षण सेटिंग्स/पैरामीटर [`ट्रेनिंगआर्ग्यूमेंट्स`](https://huggingface.co/transformers/main_classes/trainer.html# ट्रांसफॉर्मर.ट्रेनिंगआर्ग्युमेंट्स) प्रशिक्षण सेटिंग्स में वे सभी गुण होते हैं जो प्रशिक्षण प्रक्रिया को परिभाषित कर सकते हैं, इसके लिए एक फ़ोल्डर नाम की आवश्यकता होती है। इस फ़ोल्डर का उपयोग मॉडल और अन्य मॉडल कॉन्फ़िगरेशन को सहेजने के लिए किया जाएगा।

In [ ]:
args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5, #学习率
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3, # 训练的论次
    weight_decay=0.01,
)

उपरोक्त `evaluation_strategy = "epoch"` पैरामीटर प्रशिक्षण कोड को बताता है कि हम प्रति युग एक बार सत्यापन मूल्यांकन करेंगे।

उपरोक्त `बैच_आकार` इस नोटबुक से पहले परिभाषित किया गया है।

हम मॉडल में प्रीप्रोसेस्ड डेटा फीड करने के लिए default_data_collater का उपयोग करते हैं।

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

प्रशिक्षण के दौरान हम केवल नुकसान की गणना करेंगे और मूल्यांकन संकेतकों के आधार पर मॉडल का मूल्यांकन अगले भाग में करेंगे।

बस मॉडल, प्रशिक्षण पैरामीटर, डेटा, पहले उपयोग किए गए टोकननाइज़र और डेटा डिलीवरी टूल default_data_collater को ट्रानियर में पास करें।

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

प्रशिक्षण शुरू करने के लिए `ट्रेन` विधि को कॉल करें

In [ ]:
trainer.train()

चूंकि प्रशिक्षण का समय बहुत लंबा है, यदि इसे स्थानीय मैक पर प्रशिक्षित किया जाता है, तो प्रत्येक एपकोह को गायब होने में लगभग 2 सेकंड लगते हैं, इसलिए प्रत्येक प्रशिक्षण के बाद निम्नलिखित मॉडल को सहेजें।

In [ ]:
trainer.save_model("test-squad-trained")

मूल्यांकन

मॉडल मूल्यांकन थोड़ा अधिक जटिल है। हमें मॉडल आउटपुट को उस टेक्स्ट प्रारूप में पोस्ट-प्रोसेस करने की आवश्यकता है जिसकी हमें आवश्यकता है। यदि हम मूल्यांकन करते समय मॉडल को एक बैच खिलाते हैं, तो मॉडल स्वयं उत्तर की शुरुआत/अंत स्थिति पर लॉग की भविष्यवाणी करता है। आउटपुट इस प्रकार है:

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

मॉडल का आउटपुट एक तानाशाही-जैसी डेटा संरचना है, जिसमें हानि शामिल है (क्योंकि लेबल प्रदान किया गया है, इसलिए हानि है), उत्तर के लॉग प्रारंभ और अंत होते हैं, जब हम भविष्यवाणी परिणाम आउटपुट करते हैं, तो हम नहीं करते हैं नुकसान को देखने की जरूरत है, हम सीधे लॉगिट्स को देख सकते हैं।

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

प्रत्येक सुविधा में प्रत्येक टोकन में एक लॉगिट होगा। उत्तर की भविष्यवाणी करने का सबसे सरल तरीका उत्तर की प्रारंभिक स्थिति के रूप में प्रारंभ लॉग में सबसे बड़ी सबस्क्रिप्ट को चुनना है, और उत्तर की समाप्ति स्थिति के रूप में अंतिम लॉग में सबसे बड़ी सबस्क्रिप्ट को चुनना है।

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 46,  57,  78,  43, 118,  15,  72,  35,  15,  34,  73,  41,  80,  91,
         156,  35], device='cuda:0'),
 tensor([ 47,  58,  81,  55, 118, 110,  75,  37, 110,  36,  76,  53,  83,  94,
         158,  35], device='cuda:0'))

हालाँकि, उपरोक्त रणनीति ज्यादातर मामलों में अच्छी है, यदि हमारा इनपुट हमें बताता है कि हमें उत्तर नहीं मिल सकता है: उदाहरण के लिए, शुरुआत की स्थिति अंत की स्थिति से बड़ी है, या शुरुआत और अंत की स्थिति बिंदु पर है। सवाल।

इस समय, सरल तरीका यह है कि हमें अपने उत्तर के रूप में दूसरी सबसे अच्छी भविष्यवाणी को चुनना जारी रखना होगा यदि यह काम नहीं करती है, तो तीसरी सबसे अच्छी भविष्यवाणी को देखें, इत्यादि।

चूँकि उपरोक्त विधि से कोई व्यवहार्य उत्तर खोजना आसान नहीं है, इसलिए हमें और अधिक विचार करने की आवश्यकता हैउचित विधि। हम एक नया स्कोर प्राप्त करने के लिए प्रारंभ और अंत के लॉग जोड़ते हैं, और फिर सर्वोत्तम `n_best_size` प्रारंभ और अंत जोड़े को देखते हैं, `n_best_size` प्रारंभ और अंत जोड़े से, संबंधित उत्तर निकालते हैं, फिर जांचते हैं उत्तर मान्य है, और अंत में उन्हें स्कोर के अनुसार क्रमबद्ध करें, और उत्तर के रूप में उच्चतम स्कोर चुनें।

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# सर्वोत्तम आरंभ और अंत लॉग स्थान एकत्रित करें:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # 如果start小雨end，那么合理的
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # 后续需要根据token的下标将答案找出来
                }
            )

फिर हम वैध_उत्तरों को स्कोर के आधार पर क्रमबद्ध करते हैं और सर्वोत्तम उत्तर ढूंढते हैं। अंतिम चरण यह जांचना है कि प्रारंभ और अंत स्थिति के अनुरूप पाठ प्रश्न के बजाय संदर्भ में है या नहीं।

ऐसा करने के लिए, हमें सत्यापन सुविधाओं में निम्नलिखित दो जानकारी जोड़ने की आवश्यकता है:
- उस उदाहरण की आईडी जिसने सुविधा उत्पन्न की। चूंकि प्रत्येक उदाहरण कई विशेषताएं उत्पन्न कर सकता है, इसलिए प्रत्येक सुविधा/विशेषता के टुकड़े को उस उदाहरण को जानना होगा जिससे वे मेल खाते हैं।
- ऑफसेट मैपिंग: स्थिति को मैप करेंचरित्र की सबस्क्रिप्ट स्थिति के आधार पर प्रत्येक स्लाइस के टोकन को मूल पाठ में जोड़ें।

इसलिए हमने निम्नलिखित सत्यापन सेट को पुन: संसाधित किया, यह प्रशिक्षण को संसाधित करते समय `prepare_train_features` से थोड़ा अलग है।

In [ ]:

def prepare_validation_features(examples):
# हमारे उदाहरणों को ट्रंकेशन और शायद पैडिंग के साथ टोकनाइज़ करें, लेकिन एक स्ट्राइड का उपयोग करके ओवरफ़्लो बनाए रखें
# एक उदाहरण में एक संदर्भ लंबा होने पर कई विशेषताएं देना संभव है, उनमें से प्रत्येक विशेषता में एक है
# संदर्भ जो पिछली सुविधा के संदर्भ को थोड़ा ओवरलैप करता है।
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

# चूंकि एक उदाहरण हमें कई विशेषताएं दे सकता है यदि इसका संदर्भ लंबा है, तो हमें एक सुविधा से लेकर एक मानचित्र की आवश्यकता है
# इसका संगत उदाहरण यह कुंजी हमें बस यही देती है।
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

# हम example_id रखते हैं जिसने हमें यह सुविधा दी है और हम ऑफसेट मैपिंग संग्रहीत करेंगे।
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
# उस उदाहरण के अनुरूप अनुक्रम को पकड़ें (यह जानने के लिए कि संदर्भ क्या है और प्रश्न क्या है)।
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

# एक उदाहरण कई स्पैन दे सकता है, यह टेक्स्ट के इस स्पैन वाले उदाहरण का सूचकांक है।
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

# ऑफसेट_मैपिंग को कोई नहीं पर सेट करें जो संदर्भ का हिस्सा नहीं है, इसलिए यह निर्धारित करना आसान है कि कोई टोकन है या नहीं
# स्थिति संदर्भ का हिस्सा है या नहीं.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

पहले की तरह, सत्यापन सेट में प्रत्येक नमूने के लिए तैयार_मान्यता_फीचर फ़ंक्शन लागू करें।

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

सभी पूर्वानुमान परिणाम प्राप्त करने के लिए `ट्रेनर.प्रिडिक्ट` पद्धति का उपयोग करें

In [ ]:
raw_predictions = trainer.predict(validation_features)

यह `ट्रेनर` *कुछ विशेषताओं को छुपाता है* जिनका उपयोग मॉडल प्रशिक्षण के दौरान नहीं किया जाता है (यहां `example_id` और `offset_mapping` हैं, जिनका उपयोग पोस्ट-प्रोसेसिंग के दौरान किया जाएगा), इसलिए हमें उन्हें वापस सेट करने की आवश्यकता है:

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

जब टोकन स्थिति प्रश्न भाग से मेल खाती है, तो `prepare_validation_features` फ़ंक्शन ऑफ़सेट मैपिंग को `कोई नहीं` पर सेट करता है, इसलिए हम आसानी से पहचान सकते हैं कि टोकन ऑफ़सेट मैपिंग के आधार पर संदर्भ में है या नहीं। हम बहुत लंबे उत्तरों को फेंकने से भी बचते हैं .

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# पहली विशेषता पहले उदाहरण से आती है। अधिक सामान्य मामले के लिए, हमें example_id का मिलान करना होगा
# एक उदाहरण सूचकांक
context = datasets["validation"][0]["context"]

# सर्वोत्तम प्रारंभ/अंत लॉग वाले सूचकांक एकत्रित करें:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
# दायरे से बाहर उत्तरों पर विचार न करें, क्योंकि या तो सूचकांक सीमा से बाहर हैं या अनुरूप हैं
# इनपुट_आईडी के उस भाग के लिए जो संदर्भ में नहीं है।
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
# ऐसे उत्तरों पर विचार न करें जिनकी लंबाई या तो < 0 या > max_answer_length हो।
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 16.706663, 'text': 'Denver Broncos'},
 {'score': 14.635585,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 13.234194, 'text': 'Carolina Panthers'},
 {'score': 12.468662, 'text': 'Broncos'},
 {'score': 11.709289, 'text': 'Denver'},
 {'score': 10.397583,
  'text': 'Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 10.104669,
  'text': 'American Football Conference (AFC) champion Denver Broncos'},
 {'score': 9.721636,
  'text': 'The American Football Conference (AFC) champion Denver Broncos'},
 {'score': 9.007437,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10'},
 {'score': 8.834958,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina'},
 {'score': 8.38701,
  'text': 'Denver Broncos defeated the National Football Conference (NFC)'},
 {'score': 8.143825,
  'text': 'De

In [ ]:
将预测答案和真实答案进行比较即可:

In [ ]:
datasets["validation"][0]["answers"]

{'answer_start': [177, 177, 177],
 'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']}

आप देख सकते हैं कि मॉडल ने यह सही किया!

जैसा कि ऊपर दिए गए उदाहरण में बताया गया है, चूंकि पहली सुविधा पहले उदाहरण से आनी चाहिए, इसलिए यह अपेक्षाकृत आसान है। अन्य आशंकाओं के लिए, हमें सुविधाओं और उदाहरणों के बीच मैपिंग की आवश्यकता है, क्योंकि एक उदाहरण को कई विशेषताओं में विभाजित किया जा सकता है। हमें सभी सुविधाओं में सभी उत्तर एकत्र करने की भी आवश्यकता है। निम्नलिखित कोड उदाहरण की सबस्क्रिप्ट और सुविधाओं की सबस्क्रिप्ट को मैप करता है।

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

पोस्ट-प्रोसेसिंग प्रक्रिया मूल रूप से पूरी हो गई है। आखिरी बात यह है कि बिना उत्तर की स्थिति को कैसे हल किया जाए (जब स्क्वाड_v2 = सत्य)। उपरोक्त कोड केवल संदर्भ में उत्तरों पर विचार करता है, इसलिए हमें नहीं के पूर्वानुमान स्कोर भी एकत्र करने की आवश्यकता है उत्तर (CLStoken की शुरुआत और अंत बिना किसी उत्तर की भविष्यवाणी के अनुरूप है)। यदि एक उदाहरण के नमूने में कई विशेषताएं हैं, तो हमें यह भी भविष्यवाणी करने की आवश्यकता है कि क्या कई विशेषताओं में कोई उत्तर नहीं हैं, इसलिए बिना उत्तर का अंतिम स्कोर एक है वाई केसभी सुविधाओं का सबसे छोटा कोई उत्तर स्कोर नहीं।

जब तक किसी भी उत्तर का अंतिम अंक अन्य सभी उत्तरों के अंकों से अधिक नहीं होता, तब तक प्रश्न अनुत्तरित रहता है।

सब कुछ मिलाएं:

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
# इसकी संबंधित विशेषताओं के लिए एक मानचित्र उदाहरण बनाएं।
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

# जो शब्दकोष हमें भरने हैं.
    predictions = collections.OrderedDict()

#लॉगिंग.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

# आइए सभी उदाहरणों पर नज़र डालें!
    for example_index, example in enumerate(tqdm(examples)):
# वे वर्तमान उदाहरण से जुड़ी सुविधाओं के सूचकांक हैं।
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
# वर्तमान उदाहरण से जुड़ी सभी सुविधाओं के माध्यम से लूपिंग।
        for feature_index in feature_indices:
# हम इस सुविधा के लिए मॉडल की भविष्यवाणियों को पकड़ते हैं।
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
# यही वह चीज़ है जो हमें अपने लॉगिट में कुछ स्थितियों को मूल में पाठों की अवधि तक मैप करने की अनुमति देगी
# प्रसंग।
            offset_mapping = features[feature_index]["offset_mapping"]

# न्यूनतम शून्य पूर्वानुमान अपडेट करें.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

# `n_best_size` के बड़े प्रारंभ और अंत लॉग के लिए सभी संभावनाओं पर गौर करें।
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
# दायरे से बाहर उत्तरों पर विचार न करें, क्योंकि या तो सूचकांक सीमा से बाहर हैं या अनुरूप हैं
# इनपुट_आईडी के उस भाग के लिए जो संदर्भ में नहीं है।
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
# ऐसे उत्तरों पर विचार न करें जिनकी लंबाई या तो < 0 या > max_answer_length हो।
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
# अत्यंत दुर्लभ किनारे वाले मामले में हमारे पास एक भी गैर-शून्य भविष्यवाणी नहीं है, हम बचने के लिए एक नकली भविष्यवाणी बनाते हैं
# असफलता।
            best_answer = {"text": "", "score": 0.0}
        
# आइए अपना अंतिम उत्तर चुनें: सबसे अच्छा या शून्य उत्तर (केवल स्क्वाड_v2 के लिए)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

पोस्टप्रोसेसिंग फ़ंक्शन को मूल भविष्यवाणियों पर लागू करें:

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10784 features.


फिर हम मूल्यांकन संकेतक लोड करते हैं:

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

इसी प्रकार, आप लोड करने के लिए हमारे द्वारा प्रदान की गई स्थानीय स्क्रिप्ट का भी उपयोग कर सकते हैं:

In [ ]:
metric_path = './dataset/metrics/squad.py'
metric = load_metric(metric_path)

फिर हम भविष्यवाणियों और एनोटेशन के आधार पर मूल्यांकन सूचकांक की गणना करते हैं। एक उचित तुलना करने के लिए, हमें भविष्यवाणियों और एनोटेशन को प्रारूपित करने की आवश्यकता है। स्क्वाड2 के लिए, मूल्यांकन सूचकांक को पैरामीटर `no_answer_probability` की भी आवश्यकता होती है (क्योंकि कोई उत्तर नहीं है)। यह सीधे एक खाली स्ट्रिंग पर सेट है, इसलिए यह पैरामीटर यहां सीधे 0.0 पर सेट है)

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 76.74550614947965, 'f1': 85.13412652023338}

अंत में, [देखें कि मॉडल कैसे अपलोड करें](https://huggingface.co/transformers/model_sharing.html) और मॉडल को [🤗 मॉडल हब](https://huggingface.co/models) पर अपलोड करना न भूलें ) फिर आप अपने मॉडल का उपयोग नाम से कर सकते हैं, ठीक इस नोटबुक की शुरुआत की तरह।